In [1]:
# required modules
import pandas as pd
import matplotlib
import requests
from IPython.display import JSON
%matplotlib inline
matplotlib.style.use('default')
#this is a test to test if everthing is ok

In [ ]:
url = "https://api.foursquare.com/v3/places/search"
base = "46.205,6.143" #Geneva Center

headers = {
    "Accept": "application/json",
    "Authorization": ""
}

par = {
    'll':base,
    'radius':100,
    'category':13065,
    'limit':50,
    #'fields':"name,fsq_id,categories,price,rating,stats,location,geocodes,tastes"
}

response = requests.get(url, headers=headers, params=par).json()

In [ ]:
response['results']

In [ ]:
tdf = pd.json_normalize(response['results'])[[\
                                       'fsq_id',
                                       'categories',
                                       'name',
                                       'price',
                                       'rating',
                                       'stats.total_ratings',
                                       'geocodes.main.latitude',
                                       'geocodes.main.longitude',
                                       'location.formatted_address',
                                       'location.locality',
                                       #'location.neighborhood',
                                       'location.postcode',
                                       'location.region',
                                       'tastes'
                                       ]]

In [ ]:
tdf.shape

In [ ]:
def fsq(lat,lon):
    url = "https://api.foursquare.com/v3/places/search"
    #base = "46.205,6.143" #Geneva Center

    headers = {
        "Accept": "application/json",
        "Authorization": "fsq3V3VJ3Eo+Pxwu5vUFPK9qfRQiPyRblc7ovL8NXNqZIvg="
    }
    
    par = {
        'll':f'{lat},{lon}',
        'radius':100,
        'category':13065,
        'limit':50,
        'fields':"name,fsq_id,categories,price,rating,stats,location,geocodes"
    }

    response = requests.get(url, headers=headers, params=par).json()
    return pd.json_normalize(response['results'])[[\
                                                   'fsq_id',
                                                   'categories',
                                                   'name',
                                                   'price',
                                                   'rating',
                                                   'stats.total_ratings',
                                                   'geocodes.main.latitude',
                                                   'geocodes.main.longitude',
                                                   'location.formatted_address',
                                                   'location.locality',
                                                   #'location.neighborhood',
                                                   'location.postcode',
                                                   'location.region',
                                                   ]]

In [ ]:
# let's use our great df, we created in Grid Generator notebook
df_grid_points_final = pd.read_excel('../raw_data/raw_data_geneva_grid_points.xlsx', usecols='B:D') 
#df_grid_points_final

# convert grid points to numpy array for an efficient iteration which is far more efficient than iterrows()
grid_points_array = df_grid_points_final[['lat','long']].to_numpy()
print(grid_points_array.shape)
grid_points_array

In [ ]:
%%time

df_restaurants = tdf

for row in grid_points_array:
    lat, lon = row[0], row[1]
    try:
        df_restaurants = pd.concat([df_restaurants,fsq(lat,lon)])
        print(df_restaurants.shape)
    except:
        print('error')
        continue

In [ ]:
df_restaurants['categories']

In [ ]:
# Sample coordinate
lat, lng = grid_points_array[1][0], grid_points_array[1][1]
print(lat, lng)

# Sample response / Google nearby search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = {
    'keyword':'restaurant',
    'location':f'{lat},{lng}',
    'radius':100,
    'region':'ch',
    'key':google_api_access_key
}

response_sample = requests.get(base_url, params=params).json()
df_sample = pd.json_normalize(response_sample['results'])\
        [['name',
          'place_id',
          'types',
          'price_level',
          'rating',
          'user_ratings_total',
          'vicinity',
          'plus_code.global_code',
          'geometry.location.lat',
          'geometry.location.lng']]
df_sample

In [ ]:
df_restaurants.drop_duplicates()

In [ ]:
df_restaurants.drop_duplicates(subset='fsq_id',inplace=True)

In [ ]:
df_restaurants.shape

In [ ]:
df_restaurants.to_csv('../raw_data/FourSquare.csv', encoding='utf-8-sig')

### restaurant Information retrieving

In [9]:
df_id = pd.read_csv('../raw_data/4squareStats.csv', encoding='utf-8-sig')['place_id']
# df_id = df_id.head(3)
df_id.shape

(654,)

In [3]:
access_key = open('../.env/.fskey').read()                     #Tchucky's key

In [ ]:
df_id[0] = '586105ca6c682b0613bd49ec'
df_id

In [32]:
df_id = df_id[:1210]

In [10]:
c = 0
l = pd.DataFrame()
#places_list = list()
for i in df_id:
    print('\r\r\r\r',end=str(c),flush=True)
    c+=1
    
    url = f'https://api.foursquare.com/v3/places/{i}?fields=fsq_id,name,categories,tel,geocodes,rating,date_closed,stats'
    headers = {
        "Accept": "application/json",
        "Authorization": access_key
    }
#     params = {
#         'fields':'fsq_id,name,categories,tel,geocodes,rating,date_closed'
#     }

    response = requests.get(url, headers=headers, timeout=5).json()
    l = pd.concat([l,pd.json_normalize(response)])
    #places_list.append(response)

653

In [11]:
l

,fsq_id,categories,name,rating,tel,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude,stats.total_photos,stats.total_ratings,stats.total_tips
0,4e9462d36da119d220cc880b,"[{'id': 13276, 'name': 'Sushi Restaurant', 'ic...",Wasabi,6.3,022 732 36 38,46.209271,6.143641,46.209271,6.143641,20.0,63,22.0
0,4e96f87b9a528e6d36fe68d9,"[{'id': 17069, 'name': 'Grocery Store / Superm...",Coop,6.8,NaN,46.207973,6.127459,NaN,NaN,4.0,17,4.0
0,4e9b21a0f790b2a91992b62f,"[{'id': 13236, 'name': 'Italian Restaurant', '...",La Rotonde,6.5,022 732 85 59,46.210757,6.148566,46.210757,6.148566,9.0,19,7.0
0,4e9eb102e5fa0c382e117b86,"[{'id': 13032, 'name': 'Cafes, Coffee, and Tea...",La Cantinella,8.0,022 301 90 90,46.202661,6.144899,46.202661,6.144899,10.0,18,5.0
0,4e9ebc4661afec1907e721bc,"[{'id': 13032, 'name': 'Cafes, Coffee, and Tea...",Amô,5.1,022 906 11 14,46.209691,6.142241,46.209691,6.142241,13.0,15,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,4e4eade6b0fb088f3c10e65c,"[{'id': 13345, 'name': 'Spanish Restaurant', '...",Restaurant Lamartine,6.2,022 345 90 91,46.211348,6.128312,46.211348,6.128312,7.0,11,2.0
0,4e6270a3c65b2dc89fa182ee,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",El Ruedo,7.5,022 732 65 08,46.211042,6.145668,46.211042,6.145668,15.0,9,8.0
0,4e664e92d22d509a3b409d3d,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",Mogador,7.7,NaN,46.210448,6.149120,46.210448,6.149120,11.0,8,0.0
0,4e6b8460ae60950955f3d201,"[{'id': 17069, 'name': 'Grocery Store / Superm...",Coop,6.7,022 707 59 60,46.207060,6.129946,46.207060,6.129946,0.0,8,NaN


In [34]:
len(places_list)

2996

In [ ]:
df = pd.concat([df,pd.json_normalize(response)])

In [ ]:
import json
with open("test", "w") as fp:
    json.dump(places_list, fp)

In [ ]:
with open("test", "r") as fp:
    b = json.load(fp)

In [35]:
import pickle
with open("foursquarelist", "wb") as fp:   #Pickling
    pickle.dump(places_list, fp)

In [13]:
with open("test", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
b

[<Response [200]>, <Response [200]>, <Response [200]>]

In [46]:
l1 = pd.json_normalize(places_list[2500])
l2 = pd.json_normalize(places_list[2502])
#places_list[2501]

In [54]:
l = pd.DataFrame()
for i in places_list:
    l = pd.concat([l,pd.json_normalize(i)])

In [56]:
l.shape

(2996, 9)

In [12]:
l.to_csv('../raw_data/FourSquare_1.0.1.csv', encoding='utf-8-sig')

In [58]:
l.columns

Index(['fsq_id', 'categories', 'name', 'tel', 'geocodes.main.latitude',
       'geocodes.main.longitude', 'geocodes.roof.latitude',
       'geocodes.roof.longitude', 'rating'],
      dtype='object')

In [60]:
l.drop_duplicates('fsq_id', inplace=True)

In [61]:
l.shape

(2995, 9)

In [62]:
l.reset_index(inplace=True)

In [66]:
l = l[['fsq_id','categories','name','tel','geocodes.main.latitude','geocodes.main.longitude','rating']]

In [67]:
l

,fsq_id,categories,name,tel,geocodes.main.latitude,geocodes.main.longitude,rating
0,4e84bf4929c24d65fe048037,"[{'id': 13016, 'name': 'Lounge', 'icon': {'pre...",Cookfriendly Café UGS,022 735 98 98,46.201693,6.159594,NaN
1,4e85ab1177c8e4e6e12a67b4,"[{'id': 13000, 'name': 'Dining and Drinking', ...",York,NaN,46.207625,6.145186,NaN
2,4e89c104d22d940664f02037,"[{'id': 13036, 'name': 'Tea Room', 'icon': {'p...",La Mandarine Tea Room,022 700 21 70,46.202218,6.160756,NaN
3,4e8afb459adf8878b57899f6,"[{'id': 13309, 'name': 'Middle Eastern Restaur...",Hafez,022 700 00 81,46.199458,6.156029,NaN
4,4e8b395ed22dbf0a91c28b68,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",Café Tea-Room Le Cayla,022 340 06 12,46.205695,6.121689,NaN
...,...,...,...,...,...,...,...
2990,4e76074a52b1c8e519b3c9c4,"[{'id': 13145, 'name': 'Fast Food Restaurant',...",El Baraka,076 303 04 42,46.201130,6.129718,7.1
2991,4e7729172271ffa4e7ad0e7a,"[{'id': 17069, 'name': 'Grocery Store / Superm...",Coop,NaN,46.201908,6.148613,NaN
2992,4e7b22021838baac2b1836a4,"[{'id': 13302, 'name': 'Mediterranean Restaura...",Restaurant le Cazar,022 781 10 33,46.200435,6.134520,NaN
2993,4e7c6fb6a80970a1e9c128fd,"[{'id': 13352, 'name': 'Thai Restaurant', 'ico...",La Cannelle,022 328 05 42,46.199215,6.138162,NaN


In [68]:
l.to_csv('../raw_data/FourSquare_1.0.csv', encoding='utf-8-sig')

In [69]:
l.count()

fsq_id                     2995
categories                 2995
name                       2995
tel                        2367
geocodes.main.latitude     2995
geocodes.main.longitude    2995
rating                      654
dtype: int64

In [70]:
l[1]

KeyError: 1